In [22]:
import pandas as pd
import re

In [23]:
def get_senioridade(df):
    mapa_senioridade = {
        'Júnior': ['Júnior', 'Junior', 'Jr', 'Estágio', 'Estagiário', 'Trainee', 'trainne', 'I'],
        'Pleno': ['Pleno', 'Plena', 'Pl', 'Jr/Pl', 'II'],
        'Sênior': ['Sênior', 'Senior', 'Sr', 'Pl/Sr', 'Tech Lead', 'Líder', 'Especialista', 'III', 'Coordenadora']
    }

    for index, row in df.iterrows():
        for senioridade, termos in mapa_senioridade.items():
            for termo in termos:
                if re.search(r'\b' + termo + r'\b', row['titulo_da_vaga'], flags=re.IGNORECASE) or \
                   re.search(r'\b' + termo + r'\b', row['descricao'], flags=re.IGNORECASE):
                    df.at[index, 'senioridade'] = senioridade
                    break
                else:
                    df.at[index, 'senioridade'] = 'Não Informado'

    return df

In [24]:
def identificar_cargo(df):
    for index, row in df.iterrows():
        cargo = None
        
        if re.search(r'\bCientista\b', row['titulo_da_vaga'], flags=re.IGNORECASE) or \
           re.search(r'\bCientista\b', row['descricao'], flags=re.IGNORECASE) or \
           re.search(r'\bScientist\b', row['titulo_da_vaga'], flags=re.IGNORECASE) or \
           re.search(r'\bScientist\b', row['descricao'], flags=re.IGNORECASE):
            cargo = 'Cientista'
        elif re.search(r'\bAnalista\b', row['titulo_da_vaga'], flags=re.IGNORECASE) or \
             re.search(r'\bAnalista\b', row['descricao'], flags=re.IGNORECASE):
            cargo = 'Analista'
        elif re.search(r'\bEngenheir\b', row['titulo_da_vaga'], flags=re.IGNORECASE) or \
             re.search(r'\bEngenheir\b', row['descricao'], flags=re.IGNORECASE):
            cargo = 'Engenheiro'

        else:
            cargo = 'Não identificado'
            
        df.at[index, 'posicao'] = cargo

    return df

In [25]:
def identificar_regime(df):
    for index, row in df.iterrows():
        regime = None
        
        if re.search(r'\bPJ\b', row['descricao'], flags=re.IGNORECASE):
            regime = 'PJ'
        else:
            regime = 'CLT'
            
        df.at[index, 'regime'] = regime

    return df

In [26]:
def format_benefits_list(description: str) -> list[str] | None:
    """Extrai e padroniza os benefícios a partir da descrição da vaga.

    Args:
        description (str): Descrição da vaga.

    Returns:
        list[str] | None: Lista com os benefícios padronizados, ou None se não forem encontrados.

    Exemplos:
    ---
    >>> format_benefits_list('Oferecemos vale-alimentação e plano de saúde.')
    ['Vale-alimentação', 'Plano de saúde']
    """

    try:
        # Inicializa uma lista para armazenar os benefícios extraídos
        extracted_benefits = []

        # Converte toda a descrição para minúsculas para facilitar a correspondência
        description_lower = description.lower()

        # Percorre o mapa de benefícios para verificar a presença de cada benefício na descrição
        for benefit, synonyms in benefit_map.items():
            # Verifica se algum sinônimo do benefício está presente na descrição
            if any(synonym.lower() in description_lower for synonym in synonyms):
                extracted_benefits.append(benefit)

        # Retorna a lista de benefícios extraídos, removendo duplicatas e mantendo a ordem original
        if extracted_benefits:
            return list(dict.fromkeys(extracted_benefits))
        else:
            return None

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [27]:
benefit_map = {
    'Assistência médica':                    ['medica', 'saude', 'medico'],
    'Assistência odontológica':              ['odonto', 'dentista'],
    'Assistência psicológia':                ['psicolog', 'saude mental'],
    'Auxílio academia':                      ['academia', 'gympass', 'gym', 'total pass'],
    'Auxílio combustível':                   ['combustivel'],
    'Auxílio creche':                        ['creche'],
    'Auxílio desenvolvimento':               ['desenvolvimento'],
    'Auxílio estacionamento':                ['estacionamento'],
    'Auxílio farmácia':                      ['farmacia', 'medicamento'],
    'Auxílio fretado':                       ['fretado'],
    'Auxílio home office':                   ['auxilio home', 'custo home'],
    'Bicicletário':                          ['bicleta'],
    'Bolsa auxílio':                         ['bolsa auxilio'],
    'Café da manhã':                         ['cafe da manha'],
    'Cesta básica':                          ['cesta basica'],
    'Cesta de natal':                        ['natal'],
    'Clube de vantagens':                    ['vantagens'],
    'Consignado':                            ['consignado'],
    'Convênio com empresas parceiras':       ['convenio'],
    'Cooperativa de crédito':                ['cooperativa de credito'],
    'Day off aniversário':                   ['aniversario'],
    'Desconto em produtos':                  ['desconto em produtos'],
    'Ginástica laboral':                     ['ginastica laboral'],
    'Horário flexível':                      ['horario flexivel', 'flexibilidade'],
    'Massoterapia':                          ['massoterapia', 'massagem'],
    'Participação nos Lucros ou Resultados': ['lucros', 'plr', 'ppr'],
    'Plano de Aquisição de Ações':           ['aquisicao de acoes'],
    'Previdência privada':                   ['previdencia'],
    'Programa de remuneração variável':      ['remuneracao variavel', 'bonificac', 'premiac'],
    'Programa de treinamentos':              ['treinamento', 'capacitacao'],
    'Refeitório':                            ['refeitorio'],
    'Restaurante interno':                   ['restaurante'],
    'Sala de Jogos':                         ['jogos'],
    'Seguro de vida':                        ['vida'],
    'Vale-alimentação':                      ['alimentacao'],
    'Vale-cultura':                          ['vale cultura', 'vale-cultura'],
    'Vale-refeição':                         ['refeicao', 'ticket'],
    'Vale-transporte':                       ['transporte'],
}

In [28]:
df = pd.read_csv('../data/data_raw/vagas_glassdoor_raw.csv')

df.head()

,site_da_vaga,link_site,link_origem,data_publicacao,data_expiracao,data_coleta,posicao,senioridade,titulo_da_vaga,nome_empresa,cidade,estado,modalidade,contrato,regime,pcd,codigo_vaga,descricao,skills
0,Glassdoor,https://www.glassdoor.com.br/job-listing/anali...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-03-12,NaN,2024-03-13,Engenheiro de dados,NaN,Analista de Dados Pleno (Data Science).,Quod,NaN,NaN,Trabalho remoto,NaN,NaN,NaN,1009183402795,Trabalhar na área de dados na QUOD é empolgant...,"['Statistics', 'Databases', 'SQL', 'Security',..."
1,Glassdoor,https://www.glassdoor.com.br/job-listing/cient...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-03-08,NaN,2024-03-13,Engenheiro de dados,NaN,Cientista de Dados - Inteligência Artificial,"LBCA - Lee, Brock, Camargo Advogados",NaN,NaN,Trabalho remoto,NaN,NaN,NaN,1009177642279,"A LBCA é um escritório full service, formado p...","['TensorFlow', 'Statistics', 'Computer science..."
2,Glassdoor,https://www.glassdoor.com.br/job-listing/cient...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-02-28,NaN,2024-03-13,Cientista de dados,NaN,CIENTISTA DE DADOS PLENO - MONITORAMENTO (vaga...,ClearSale,NaN,NaN,Trabalho remoto,NaN,NaN,NaN,1009147683343,JOB DESCRIPTIONA Área de Modelagem da ClearSal...,"['Statistics', 'E-commerce', 'SQL', 'Statistic..."
3,Glassdoor,https://www.glassdoor.com.br/job-listing/cient...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-03-05,NaN,2024-03-13,Cientista de dados,NaN,Cientista de Dados - Vaga Afirmativa para Mulh...,"Vagas, empregos, oportunidades",NaN,NaN,Trabalho remoto,NaN,NaN,NaN,1009159657667,Na posição de Cientista de Dados na Conta Azul...,"['Statistics', 'SQL', 'Math', 'Machine learnin..."
4,Glassdoor,https://www.glassdoor.com.br/job-listing/cas-c...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-03-02,NaN,2024-03-13,Engenheiro de dados,NaN,CAS | Cientista de Dados - Foco em Crédito,Sicredi,NaN,NaN,Trabalho remoto,NaN,NaN,NaN,1009055258960,Integrar os times de dados da Diretoria Execut...,"['Statistics', 'Cloud infrastructure', 'Relati..."


In [29]:
df = get_senioridade(df)
df = identificar_cargo(df)
df = identificar_regime(df)
df['senioridade'].replace('NaN', 'Não informado')
df['modalidade'] = df['modalidade'].replace('trabalho remoto', 'Remoto')
df['modalidade'] = df['modalidade'].fillna('Presencial')
df['estado'] = df['estado'].replace('Distrito Federal', 'DF')
df['modalidade'] = df['modalidade'].replace('Trabalho remoto','Remoto')
df.loc[df['modalidade'] == 'Remoto', 'cidade'] = 'Não informado'
df.loc[df['modalidade'] == 'Remoto', 'estado'] = 'Todo o Brasil'

df['beneficios'] = df['descricao'].apply(format_benefits_list)

In [30]:
df.head()

,site_da_vaga,link_site,link_origem,data_publicacao,data_expiracao,data_coleta,posicao,senioridade,titulo_da_vaga,nome_empresa,cidade,estado,modalidade,contrato,regime,pcd,codigo_vaga,descricao,skills,beneficios
0,Glassdoor,https://www.glassdoor.com.br/job-listing/anali...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-03-12,NaN,2024-03-13,Analista,Não Informado,Analista de Dados Pleno (Data Science).,Quod,Não informado,Todo o Brasil,Remoto,NaN,CLT,NaN,1009183402795,Trabalhar na área de dados na QUOD é empolgant...,"['Statistics', 'Databases', 'SQL', 'Security',...","[Assistência odontológica, Auxílio academia, A..."
1,Glassdoor,https://www.glassdoor.com.br/job-listing/cient...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-03-08,NaN,2024-03-13,Cientista,Não Informado,Cientista de Dados - Inteligência Artificial,"LBCA - Lee, Brock, Camargo Advogados",Não informado,Todo o Brasil,Remoto,NaN,CLT,NaN,1009177642279,"A LBCA é um escritório full service, formado p...","['TensorFlow', 'Statistics', 'Computer science...","[Auxílio creche, Auxílio desenvolvimento, Prog..."
2,Glassdoor,https://www.glassdoor.com.br/job-listing/cient...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-02-28,NaN,2024-03-13,Cientista,Não Informado,CIENTISTA DE DADOS PLENO - MONITORAMENTO (vaga...,ClearSale,Não informado,Todo o Brasil,Remoto,NaN,CLT,NaN,1009147683343,JOB DESCRIPTIONA Área de Modelagem da ClearSal...,"['Statistics', 'E-commerce', 'SQL', 'Statistic...","[Assistência médica, Assistência odontológica,..."
3,Glassdoor,https://www.glassdoor.com.br/job-listing/cient...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-03-05,NaN,2024-03-13,Cientista,Não Informado,Cientista de Dados - Vaga Afirmativa para Mulh...,"Vagas, empregos, oportunidades",Não informado,Todo o Brasil,Remoto,NaN,CLT,NaN,1009159657667,Na posição de Cientista de Dados na Conta Azul...,"['Statistics', 'SQL', 'Math', 'Machine learnin...","[Assistência médica, Assistência odontológica,..."
4,Glassdoor,https://www.glassdoor.com.br/job-listing/cas-c...,www.glassdoor.com.br/Vaga/index.htm/partner/jo...,2024-03-02,NaN,2024-03-13,Cientista,Não Informado,CAS | Cientista de Dados - Foco em Crédito,Sicredi,Não informado,Todo o Brasil,Remoto,NaN,CLT,NaN,1009055258960,Integrar os times de dados da Diretoria Execut...,"['Statistics', 'Cloud infrastructure', 'Relati...","[Assistência odontológica, Auxílio creche, Aux..."


In [31]:
df.to_csv('../data/data_clean/vagas_glassdoor_clean.csv', index=False)